In [ ]:
import boto3
import json
import numpy as np


def respond(err, res=None):
    ret = {
        "statusCode": "400" if err else "200",
        "body": err.message if err else json.dumps(res),
        "headers": {
            "Content-Type": "application/json",
        },
    }
    return json.dumps(ret)


"""
input:
{
  "inputTexts": ["Hello World!", "Hello Nordea!"],
  "operation": "GetEmbeddings"
}

output:
{
    "embeddings": [[0.1, 0.2, 0.3], [0.4, 0.5, 0.6]],
    "operation": "GetEmbeddings",
}
"""

client = boto3.client("bedrock-runtime")
accept = "application/json"
contentType = "application/json"

operation2ModelId = {
    "GetEmbeddings": "amazon.titan-embed-text-v1",
    "GetEmbeddingsAndSimilarity": "amazon.titan-embed-text-v1",
}


def processEmbedding(event):
    operation = event["operation"]
    modelId = operation2ModelId[operation]

    # invoke model a few times to get embeddings for all input texts
    inputTexts = event["inputTexts"]
    embeddings = []
    for inputText in inputTexts:
        body = json.dumps(
            {
                "inputText": inputText,
            }
        )

        response = client.invoke_model(
            body=body, modelId=modelId, accept=accept, contentType=contentType
        )
        response_body = json.loads(response.get("body").read())

        embeddings.append(response_body["embedding"])

    response_dict = {"embeddings": embeddings, "operation": operation}

    response = respond(None, response_dict)
    return response

def str2List(string, seperate_char="@"):
    return string.split(seperate_char)
def processEmbeddingAndSimilarity(event):
    operation = event["operation"]
    modelId = operation2ModelId[operation]

    # invoke model a few times to get embeddings for all input texts
    inputTexts = event["inputTexts"]
    embeddings = []
    for inputText in inputTexts:
        body = json.dumps(
            {
                "inputText": inputText,
            }
        )

        response = client.invoke_model(
            body=body, modelId=modelId, accept=accept, contentType=contentType
        )
        response_body = json.loads(response.get("body").read())

        embeddings.append(response_body["embedding"])
    
    # calculate similarity
    textNum = len(embeddings)
    similarity = np.eye(textNum)
    for i in range(textNum):
        for j in range(i+1, textNum):
            similarity[i][j] = np.dot(embeddings[i], embeddings[j]) / (np.linalg.norm(embeddings[i]) * np.linalg.norm(embeddings[j]))
            similarity[j][i] = similarity[i][j]

    distance = 1 - similarity

    response_dict = {"embeddings": embeddings, "similarity": similarity.tolist(), "operation": operation, "distance": distance.tolist()}
    
    response = respond(None, response_dict)
    return response
    

def lambda_handler(event, context):
    print("Received event: " + json.dumps(event, indent=2))

    if not "operation" in event:
        event = event["queryStringParameters"]
        event["inputTexts"] = str2List(event["inputTexts"])
    
    operation = event["operation"]
    
    if operation == "GetEmbeddings":
        return processEmbedding(event)
    elif operation == "GetEmbeddingsAndSimilarity":
        return processEmbeddingAndSimilarity(event)
    else:
        return respond(Exception("Invalid operation"), None)
    

In [43]:
import requests
import json

# The URL of the API endpoint
url = 'https://fdbhhaqp4i.execute-api.us-west-2.amazonaws.com/NordXDev/'

seperate_char = "@"

inputTexts = ["Hello World!", "Hello Nordea!"]
inputTextsStr = seperate_char.join(inputTexts)

# Optional: Parameters to send with the request (if needed)
params = {
    # "body": {
  "inputTexts": inputTextsStr,
  "operation": "GetEmbeddingsAndSimilarity"
}

# Making a GET request
response = requests.get(url, params=params)

print(response)
# Check if the request was successful
if response.status_code == 200:
    # Process the response data (if response is JSON)
    data = response.json()['body']
    print(data)
else:
    print("Failed to retrieve data:", response.status_code)

<Response [200]>
{"embeddings": [[0.45703125, 0.30078125, 0.41210938, 0.41015625, 0.74609375, 0.3828125, -0.018798828, -0.0010681152, 0.515625, -0.25195312, -0.20898438, -0.15332031, -0.22460938, 0.13671875, -0.34375, 0.037109375, -0.03491211, -0.29882812, -0.875, 0.21289062, -0.39648438, 0.26171875, -0.49609375, -0.15527344, -0.014465332, 0.25, -0.1796875, -0.46875, 0.82421875, 0.1796875, -0.34960938, 1.2265625, -0.079589844, -0.3046875, -0.93359375, 0.3828125, -0.031982422, -0.74609375, -0.49023438, 0.546875, 0.080078125, 0.328125, -0.1484375, -0.30078125, 0.54296875, 0.58203125, -0.1875, -0.17773438, 0.59375, 1.0859375, -0.20703125, 0.99609375, -0.25195312, 0.6875, -0.10839844, -0.11035156, 0.66015625, -0.12695312, -0.0079956055, -0.37695312, -0.038085938, -0.34570312, -0.62890625, 0.94921875, 0.12597656, 0.125, -0.12109375, 0.3515625, -0.17285156, 0.6328125, 0.41210938, 0.48828125, 0.19140625, 0.6640625, -0.67578125, 0.23730469, 0.80859375, -0.16796875, 0.43164062, 0.27148438, 0.42